In [1]:
import requests
import json
import pandas as pd
from tqdm import tqdm

In [29]:
headers = {"User-Agent": "Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/114.0.0.0 Safari/537.36",
}
params = {
    "limit": "48", # limit number of items each page
    "include": "advertisement",
    "aggregation": "2",
    "category": "1846",
    #"page": "1", # page number
    "urlkey": "laptop-may-vi-tinh-linh-kien" # product category
}
new_api_request_url = "https://tiki.vn/api/v2/products" # new api from tiki developer
old_api_request_url = "https://tiki.vn/api/personalish/v1/blocks/listings" # current api from tiki webpage
#pageNum = 1
product_id_list = []
df_id = pd.DataFrame([], columns=["id"])
# old api page: 2000 - 209   2000 - 209
# old api no page: 47376 - 988
# new-api-page : 2000 - 198  2000 - 199  2000 - 240 
# new-api-no page:  10080 - 211    8400 - 176

<b>Fetch Product ID Data<b>

In [30]:
while True:       
    '''With the page number param available, data return only 2000 items -> I will block it to get as much data as possible''' 
    #print("Crawl page number: ", pageNum)
    #params["page"] = pageNum
    response = requests.get(new_api_request_url, headers=headers, params=params)
    if response.status_code == 200:
        #print("Crawling data successfully")
        #products = json.loads(response.text)["data"]
        for product in response.json()["data"]:
            id = str(product["id"])
            product_id_list.append(id)
    else:
        print(f"Error, status code = {response.status_code}")
        break

    # Divided each batch with maximum 2000 items 
    if(len(product_id_list) % 2000 == 0):
        print("Reach 2000 items")
        df_temp = pd.DataFrame(product_id_list, columns=["id"])
        df_id = pd.concat([df_id, df_temp], ignore_index=True)
        # Truncate all the list
        del product_id_list[:]
        print(len(product_id_list))
        
    #pageNum+=1

Reach 2000 items
0


JSONDecodeError: Expecting value: line 1 column 1 (char 0)

In [31]:
print(len(product_id_list))
df_id.shape

4320


(6000, 1)

In [32]:
# Fill out the list that not reach 2000 items
df_temp = pd.DataFrame(product_id_list, columns=["id"])
df_id = pd.concat([df_id, df_temp], ignore_index=True)
# Truncate all the list
del product_id_list[:]
print(len(product_id_list))
df_id.shape

0


(10320, 1)

In [33]:
df_id.head()

,id
0,38323087
1,26314426
2,134561311
3,8608925
4,46634155


In [34]:
list_temp = df_id["id"].values.tolist()

In [35]:
print(len(list_temp))

10320


<b> Fetch Product Details Data <b>

In [75]:
url = "https://tiki.vn/api/v2/products/{}"
pars = {"platform": "web", 
        "spid": "124939771"
}
product_detail_list = []
product_json_list = []
df_product = pd.DataFrame([])

In [76]:
for id in tqdm(list_temp, total=len(list_temp)):
    response = requests.get('https://tiki.vn/api/v2/products/{}'.format(id), headers=headers, params=pars)
    if(response.status_code==200):
        # Extract value attributes
        # product["id"] = response.json()['id']
        # product["name"] = response.json()['name']
        # product["sku"] = response.json()['sku']
        # product["price"] = response.json()['price']
        # product["list_price"] = response.json()['list_price']
        # product["discount"] = response.json()['discount']
        # product["discount_rate"] = response.json()['discount_rate']
        # product["inventory_status"] = response.json()['inventory_status']
        # product["stock_item_max_sale_qty"] = response.json()['stock_item']['max_sale_qty']
        # product["stock_item_qty"] = response.json()['stock_item']['qty']
        # product["qty_sold"] = response.json()['all_time_quantity_sold']
        # product["brand_id"] = response.json()['brand']['id']
        # product["brand_name"] = response.json()['brand']['name']
        # product_detail_list.append(product)
        #----------------
        try:
            #product_detail_list.append(response.text)
            product = json.loads(response.text)         
            product_detail_list.append(product)
        except:
            #print("Something else went wrong")
            pass

    else:
        print(f"Error, status code = {response.status_code}")
        pass

    # Divided each batch with maximum 2000 items 
    if(len(product_detail_list) == 2000):
        print("Reach 2000 items")
        # product_json_list = [adjust_product(p) for p in product_detail_list]
        # df_temp = pd.DataFrame.from_dict(product_json_list)
        df_temp = pd.DataFrame.from_dict(product_detail_list)
        df_product = pd.concat([df_product, df_temp], ignore_index=True)
        # Truncate all the list
        del product_detail_list[:]
        del product_json_list[:]
        print(len(product_detail_list), len(product_json_list))
    
        

 30%|███       | 3120/10320 [16:14<36:55,  3.25it/s]  

Reach 2000 items
0 0


 62%|██████▏   | 6446/10320 [33:05<22:42,  2.84it/s]  

Reach 2000 items
0 0


 94%|█████████▍| 9738/10320 [49:49<03:35,  2.70it/s]

Reach 2000 items
0 0


100%|██████████| 10320/10320 [52:52<00:00,  3.25it/s]


In [78]:
print(len(product_detail_list))
df_product.shape

367


(6000, 70)

In [80]:
# Fill out the list that not reach 2000 items
df_temp = pd.DataFrame.from_dict(product_detail_list)
df_product = pd.concat([df_product, df_temp], ignore_index=True)
# Truncate all the list
del product_detail_list[:]
print(len(product_detail_list))
df_product.shape

0


(6367, 70)

In [81]:
df_product.head()

,id,master_id,sku,name,url_key,url_path,short_url,type,book_cover,short_description,...,inventory,warranty_info,return_and_exchange_policy,is_tier_pricing_available,is_tier_pricing_eligible,asa_cashback_widget,benefits,price_comparison,video_url,best_price_guaranteed
0,38323087,38323087,9386902161867,(Giá Hủy Diệt) Ổ Cứng Di Động HDD WD My Passpo...,o-cung-di-dong-wd-my-passport-model-2019-hang-...,o-cung-di-dong-wd-my-passport-model-2019-hang-...,https://tiki.vn/product-p38323087.html?spid=53...,configurable,None,Thông số kỹ thuật WD My Passport màu đen mode...,...,"{'product_virtual_type': None, 'fulfillment_ty...","[{'name': 'Thời gian bảo hành', 'value': '36 T...",Đổi trả trong<br><b>7 ngày</b><br>nếu sp lỗi.,False,False,"{'sep': 0, 'rate': 213, 'astra_convert_rate': ...",[{'icon': 'https://salt.tikicdn.com/ts/upload/...,NaN,NaN,NaN
1,26314426,26314426,9365508570174,Dây Cáp Mạng Cat6 Bấm Sẵn 2 đầu,day-cap-mang-cat6-bam-san-2-dau-p26314426,day-cap-mang-cat6-bam-san-2-dau-p26314426.html...,https://tiki.vn/product-p26314426.html?spid=14...,configurable,None,+ Cáp mạng CAT6 UTP được sử dụng chất liệu CCA...,...,"{'product_virtual_type': None, 'fulfillment_ty...","[{'name': 'Hướng dẫn bảo hành', 'value': 'Xem ...",Đổi trả trong<br><b>7 ngày</b><br>nếu sp lỗi.,False,False,"{'sep': 0, 'rate': 213, 'astra_convert_rate': ...",[{'icon': 'https://salt.tikicdn.com/ts/upload/...,"{'sub_title': 'Giá từ 32.650 ₫', 'title': '6 n...",NaN,NaN
2,134561311,134561311,8367091212771,USB 3.0 SanDisk Ultra Flair CZ73 - Hàng Nhập Khẩu,usb-3-0-sandisk-ultra-flair-cz73-64gb-hang-nha...,usb-3-0-sandisk-ultra-flair-cz73-64gb-hang-nha...,https://tiki.vn/product-p134561311.html?spid=9...,configurable,None,Phần mềm bảo vệ và chống truy cập trái phép Se...,...,"{'product_virtual_type': None, 'fulfillment_ty...","[{'name': 'Thời gian bảo hành', 'value': '5 Nă...",Đổi trả trong<br><b>7 ngày</b><br>nếu sp lỗi.,False,False,"{'sep': 0, 'rate': 213, 'astra_convert_rate': ...",[{'icon': 'https://salt.tikicdn.com/ts/upload/...,NaN,NaN,NaN
3,8608925,8608925,6460064428402,Hộp mực dành cho máy in Laser Brother HL-L2321D.,hop-muc-may-in-laser-brother-hl-l2321d-p8608925,hop-muc-may-in-laser-brother-hl-l2321d-p860892...,https://tiki.vn/product-p8608925.html?spid=263...,simple,None,Hộp mực TN-2385/2325... hàng thay thế thích hợ...,...,"{'product_virtual_type': None, 'fulfillment_ty...","[{'name': 'Thời gian bảo hành', 'value': '1 Th...",Đổi trả trong<br><b>7 ngày</b><br>nếu sp lỗi.,False,False,"{'sep': 0, 'rate': 213, 'astra_convert_rate': ...",[{'icon': 'https://salt.tikicdn.com/ts/upload/...,"{'sub_title': 'Giá từ 107.900 ₫', 'title': '9 ...",NaN,NaN
4,46634155,46634155,7172023663098,Dây Cáp Mạng Cat6 Bấm Sẵn 2 đầu,day-cap-mang-cat6-bam-san-2-dau-p46634155,day-cap-mang-cat6-bam-san-2-dau-p46634155.html...,https://tiki.vn/product-p46634155.html?spid=16...,configurable,None,Vỏ bọc bên ngoài làm bằng nhựa PVC nguyên chất...,...,"{'product_virtual_type': None, 'fulfillment_ty...","[{'name': 'Hướng dẫn bảo hành', 'value': 'Xem ...",Đổi trả trong<br><b>7 ngày</b><br>nếu sp lỗi.,False,False,"{'sep': 0, 'rate': 213, 'astra_convert_rate': ...",[{'icon': 'https://salt.tikicdn.com/ts/upload/...,"{'sub_title': 'Giá từ 24.900 ₫', 'title': '2 n...",NaN,NaN


In [82]:
df_trans = df_product[['id', 'name', 'sku', 'price', 'list_price', 'discount', 'discount_rate', 'inventory_status', 'all_time_quantity_sold']]
df_trans.head()

,id,name,sku,price,list_price,discount,discount_rate,inventory_status,all_time_quantity_sold
0,38323087,(Giá Hủy Diệt) Ổ Cứng Di Động HDD WD My Passpo...,9386902161867,1385000,1390000,5000,0,available,2244
1,26314426,Dây Cáp Mạng Cat6 Bấm Sẵn 2 đầu,9365508570174,75000,75000,0,0,available,1120
2,134561311,USB 3.0 SanDisk Ultra Flair CZ73 - Hàng Nhập Khẩu,8367091212771,110000,110000,0,0,available,9979
3,8608925,Hộp mực dành cho máy in Laser Brother HL-L2321D.,6460064428402,111900,111900,0,0,available,2638
4,46634155,Dây Cáp Mạng Cat6 Bấm Sẵn 2 đầu,7172023663098,48000,48000,0,0,available,490


In [83]:
# stock_item: max_sale_qty, qty
# brand: id, name
df_stock = df_product['stock_item'].apply(pd.Series)
df_brand = df_product['brand'].apply(pd.Series)
df_stock = df_stock[['max_sale_qty', 'qty']]
df_brand = df_brand[['id', 'name']]
df_brand.rename(columns={'id': 'brand_id', 'name': 'brand_name'}, inplace=True)

In [84]:
df_final = pd.concat([df_trans, df_stock, df_brand], axis=1)
df_final.head()

,id,name,sku,price,list_price,discount,discount_rate,inventory_status,all_time_quantity_sold,max_sale_qty,qty,brand_id,brand_name
0,38323087,(Giá Hủy Diệt) Ổ Cứng Di Động HDD WD My Passpo...,9386902161867,1385000,1390000,5000,0,available,2244,1000.0,1000.0,21862,WD
1,26314426,Dây Cáp Mạng Cat6 Bấm Sẵn 2 đầu,9365508570174,75000,75000,0,0,available,1120,1000.0,1000.0,111461,OEM
2,134561311,USB 3.0 SanDisk Ultra Flair CZ73 - Hàng Nhập Khẩu,8367091212771,110000,110000,0,0,available,9979,1000.0,1000.0,18845,SanDisk
3,8608925,Hộp mực dành cho máy in Laser Brother HL-L2321D.,6460064428402,111900,111900,0,0,available,2638,1000.0,1000.0,111461,OEM
4,46634155,Dây Cáp Mạng Cat6 Bấm Sẵn 2 đầu,7172023663098,48000,48000,0,0,available,490,1000.0,1000.0,111461,OEM


In [91]:
df_product.to_csv("product_full_details.csv", encoding='utf-8-sig')
df_final.to_csv("product_short_detials.csv", encoding='utf-8-sig')